In [4]:
import pickle
import os
def f(dict, path):
    path = os.path.join(path, dict)
    with open(path, 'rb') as f:
        return pickle.load(f)

path = os.path.join("emoji_data")
files = os.listdir(path)
loaded_dicts = [f(file, path) for file in files]

In [5]:
dics_names = [
    "EMOJI_UNICODE",
    "UNICODE_EMOJI",
    "EMOJI_ALIAS_UNICODE",
    "UNICODE_EMOJI_ALIAS",
    "EMOTICONS_EMO",
]

In [6]:
import numpy as np
import pandas as pd

In [7]:
keys = np.array(list(loaded_dicts[2].keys()))


In [8]:
def find_indeces(columns, values):
    columns = columns.flatten()
    inds = np.array([np.where(columns == searchval)[0] for searchval in values]).flatten()
    return inds

In [14]:
main = pd.read_csv("dataset_.csv")[["emoi","label"]]

dropped_empty = main[main.emoi != "[]"]
import ast
dropped_empty['emoi']=dropped_empty['emoi'].apply(ast.literal_eval)
emojis_set = dropped_empty.emoi.to_list()
encoded = np.zeros((len(emojis_set),keys.shape[0]))
for row, emojis in zip(encoded, emojis_set):
    row[find_indeces(keys,emojis)] = 1
encoded_df = pd.DataFrame(encoded)
dropped_empty.reset_index(drop=True, inplace=True)
dropped_empty.index
merged = dropped_empty.join(encoded_df)
merged.drop(columns="emoi", inplace=True)

/var/folders/4_/s_rxdxsd00j5rpk8jz76d7mh0000gn/T/ipykernel_58893/3860012720.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped_empty['emoi']=dropped_empty['emoi'].apply(ast.literal_eval)


In [15]:
merged

,label,0,1,2,3,4,5,6,7,8,...,3536,3537,3538,3539,3540,3541,3542,3543,3544,3545
0,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2583,disappointed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2584,disappointed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2585,disappointed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2586,disappointed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
y = merged.label

0              happy
1              happy
2              happy
3              happy
4              happy
            ...     
2583    disappointed
2584    disappointed
2585    disappointed
2586    disappointed
2587    disappointed
Name: label, Length: 2588, dtype: object